<a href="https://colab.research.google.com/github/durga41003/Data-Science/blob/main/Fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.6 MB/s eta 0:00:00


In [ ]:
from faker import Faker
import random
import csv
f=Faker()

In [ ]:
data=[]
for _ in range(6000):
  state=f.text(max_nb_chars=100)
  data.append((state,1))

In [ ]:
for _ in range(5000):
  state=f.text(max_nb_chars=100)
  data.append((state,0))


In [ ]:
random.shuffle(data)

In [ ]:
with open('fake_news_data.csv','w',newline='') as file:
  writer=csv.writer(file)
  writer.writerow(['state','label'])
  writer.writerows(data)

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("fake_news_data.csv")
df

,state,label
0,Condition fire father. Wall simply probably co...,0
1,Million dark establish office. Audience bed be...,1
2,Teach practice drug myself there also.,1
3,Season member international how loss.,1
4,Car commercial section value many recognize.,0
...,...,...
10995,Agree say this car pattern program door.,1
10996,Into current son industry. Hear figure modern.,1
10997,Red who go agree. Else site none book among ma...,1
10998,Prevent later send show each paper series. Hai...,1


In [ ]:
df.head()

,state,label
0,Condition fire father. Wall simply probably co...,0
1,Million dark establish office. Audience bed be...,1
2,Teach practice drug myself there also.,1
3,Season member international how loss.,1
4,Car commercial section value many recognize.,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   11000 non-null  object
 1   label   11000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 172.0+ KB


In [ ]:
df.describe()

,label
count,11000.000000
mean,0.545455
std,0.497952
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
df.isnull().sum()

state    0
label    0
dtype: int64

In [ ]:
df['label'].value_counts()

1    6000
0    5000
Name: label, dtype: int64

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [ ]:
from sklearn import datasets
from nltk.stem import WordNetLemmatizer
le= WordNetLemmatizer()
from nltk.corpus import stopwords
import re
dataset=[]
for i in range(0,len(df)):
  review=re.sub('[^a-zA-Z]'," ", df["state"][i])
  review=review.lower()
  review=review.split()
  review=[le.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review=" ".join(review)
  dataset.append(review)

In [ ]:
#!pip install TfidVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tv= TfidfVectorizer(max_features=2500)
X=tv.fit_transform(dataset).toarray()
Y=df.label

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0,stratify=Y)

In [ ]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8800, 867) (8800,)
(2200, 867) (2200,)


In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,Y_train)

LogisticRegression()

In [ ]:
lr_ans=lr.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
score1=accuracy_score(Y_test,lr_ans)
print(score1)

0.49727272727272726


In [ ]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(X_train,Y_train)

SVC()

In [ ]:
svc_ans = svc.predict(X_test)
score2= accuracy_score(Y_test,svc_ans)
print(score2)

0.5063636363636363


In [ ]:
input_news=["Team members said that they won"]
input_news = tv.transform(input_news)
prediction=lr.predict(input_news)
print(prediction)
if(prediction[0]==1):
  print("True news")
else:
  print("False news")

[1]
True news
